In [18]:
import math
import pyprob
from pyprob import Model
from pyprob.distributions import Normal, Uniform

pyprob.set_verbosity(1)

## PyProb 

* **PyProb** is a PyTorch-based probabilistic programming language
* It supports **inference compilation** with minimal intervention
* It uses a protocol (PPX) that allows **distributed** training and inference

## Primitive Stochastic Functions

* Built on top of **PyTorch's** distributions library
* API is, however, slightly different than Pyro's

### Drawing sample from unit Normal 

In [13]:
mu = 0.
sigma = 1.

In [14]:
normal = Normal(mu, sigma) # create a normal distribution object

In [15]:
x = normal.sample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor(-0.3696)
log prob tensor(-0.9873)


* Just like in the Pyro examples, these distributions and sampling functions are backed by the underlying PyTorch implementation.

### Sampling 

In [16]:
x = Normal(mu, sigma).sample()
print(x)

tensor(-0.2404)


* No need to explicitly pass a name to the random variable.

## The Ice-Cream Model

Let’s suppose we have a bunch of data with daily mean temperatures and cloud cover. We want to reason about how temperature interacts with whether it was sunny or cloudy. A simple stochastic function that does that is given by:

In [40]:
def weather():
    cloudy = Uniform(0, 1).sample()
    cloudy = 'cloudy' if cloudy < 0.3 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = Normal(mean_temp, scale_temp).sample()
    return cloudy, temp
    
for _ in range(3):
    print(weather())

('sunny', tensor(83.3104))
('cloudy', tensor(62.5814))
('cloudy', tensor(47.2370))


In [45]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    return Normal(expected_sales, 10.0).sample()

In [46]:
ice_cream_sales()

tensor(186.1586)

# Inference: One Example

In [47]:
class GaussianUnknownMean(Model):
    def __init__(self):
        super().__init__(name="Gaussian with unknown mean") # give the model a name
        
        # prior mean and std
        self.prior_mean = 1
        self.prior_stdd = math.sqrt(5)
        self.likelhood_stdd = math.sqrt(2)

    # Needed to specifcy how the generative model is run forward
    def forward(self):
        # sample the (latent) mean variable to be inferred:
        mu = pyprob.sample(Normal(self.prior_mean, self.prior_stdd))

        # define the likelihood
        likelihood = Normal(mu, self.likelhood_stdd)


        # observation "placeholders"
        # these can have concrete values associated with them later when
        # conditioning on data.
        pyprob.observe(likelihood, name='observation-1')
        pyprob.observe(likelihood, name='observation-2')

        # return the latent quantity of interest
        return mu

# create an instance of the model
gum = GaussianUnknownMean()

* The probabilistic program above models **Gaussian with Unknown Mean**: our goal is to infer the expected value of `mu` above.
* PyProb supports multiple inference algorithms, including inference compilation (discussed later)

In [66]:
posterior = gum.posterior_distribution(
    num_traces=1000, # number of samples to be used
    inference_engine=pyprob.InferenceEngine.IMPORTANCE_SAMPLING, # the inference algorithm
    observe={'observation-1': 9, 'observation-2': 8.4} # conditioning on some observations
)

# inferring the mean and standard deviation of our `mu` latent variable
print("mean:", posterior.mean)
print("stddev:", posterior.stddev)

done
mean: tensor(7.1997)
stddev: tensor(0.6403)


* PyProb provides other inference algorithms too:
    * Importance sampling
    * Lightweight Metropolis-Hastings
    * Random Walk Metropolis-Hastings

# Inference Compilation

PyProb supports **inference compilation**. Let's apply that to the _Gaussian with Unknown Mean_ example we used in the last section. Recall that we defined a model for it:

In [60]:
gum

PyProb provides an inference engine to use in combination with a **learned inference network**.

First, we need to learn the inference network:

In [54]:
gum.learn_inference_network(
    num_traces=1000, # how many traces (executions of the generative model) to use to train the inference network
    observe_embeddings={
        'observation-1': {'dim': 10},
        'observation-2': {'dim': 10}
    }
)

Creating new inference network...
Initializing inference network...
Observable observation-1: observe embedding not specified, using the default FEEDFORWARD.
Observable observation-1: embedding depth not specified, using the default 2.
Observable observation-2: observe embedding not specified, using the default FEEDFORWARD.
Observable observation-2: embedding depth not specified, using the default 2.
New proposal layer for address: 16__forward__mu__Normal__1
Total number of parameters: 5,834
Train. time | Trace     | Init. loss| Min. loss | Curr. loss| T.since min | Traces/sec
0d:00:00:02 | 1,024     | +2.26e+00 | +2.10e+00 | +2.26e+00 | 0d:00:00:01 | 593.0                              


Once the inference network is **trained** with the traces produced by the generative model, we can use it to perform **inference**:

In [61]:
posterior = gum.posterior_distribution(
    num_traces=1000,
    inference_engine=pyprob.InferenceEngine.IMPORTANCE_SAMPLING_WITH_INFERENCE_NETWORK,
    observe={'observation-1': 9, 'observation-2': 8.4}
)

# inferring the mean and standard deviation of our `mu` latent variable
print("mean:", posterior.mean)
print("stddev:", posterior.stddev)

mean: tensor(7.2872)
stddev: tensor(0.7825)
